# <b> Buy and Hold </b>

Example showing the backtesting of a Buy and Hold strategy.

When the strategy starts, it will buy 50% of AAPL and 50% of GOOG.
As soon as the backtest completes, it will return a dataframe 'result' containing the results, which can be used to visualize the equity's evolution and other metrics, like when buy or sell orders are placed.

## <b> Import Libraries </b>

In [1]:
# Import this library
from nbacktest import Backtest, Strategy

# Import other libraries
import yfinance as yf
import warnings

# Ignore warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## <b> Create Strategy </b>

In [2]:
class TestStrategy (Strategy):

    def on_start(self):

        print("Strategy started!")
        
        quantity1 = int((self.balance/2)/self.prices["AAPL"])
        quantity2 = int((self.balance/2)/self.prices["GOOG"])

        order1 = self.buy("AAPL", quantity1, notes="AAPL buy order")
        order2 = self.buy("GOOG", quantity2, notes="GOOG buy order")

        trade1 = self.create_trade(orders=[order1, order2], notes="Buy and Hold Trade")

        print("Bought %s AAPL and %s GOOG" % (quantity1, quantity2))


    def on_end(self):
        """"
        This function is called when the backtest of all candles is finished
        """
        print("Strategy finished!")


    def next(self):
        """"
        This function is called every candle (iteration)
        """
        #print("Backtest iteration:", self.iteration)
        pass


## <b> Download OHLC data from Yahoo Finance </b>

In [3]:
universe = ["AAPL", "GOOG"] # List containing the tickers of all the assets you are backtesting

df_ohlc = yf.download(universe, start="2018-01-01", end="2023-01-01", interval="1d") # Download df_ohlc data from Yahoo Finance

print(df_ohlc.shape)
df_ohlc.tail()

[*********************100%***********************]  2 of 2 completed

(1259, 10)


Price            Close                   High                    Low  \
Ticker            AAPL       GOOG        AAPL       GOOG        AAPL   
Date                                                                   
2022-12-23  129.900299  89.141426  130.451974  89.429268  127.713291   
2022-12-27  128.097504  87.275421  129.456999  88.833733  126.806975   
2022-12-28  124.166794  85.816353  129.082630  87.861015  123.999322   
2022-12-29  127.683716  88.287827  128.540781  88.699738  125.831659   
2022-12-30  127.998985  88.069473  128.018692  88.168727  125.536148   

Price                        Open               Volume            
Ticker           GOOG        AAPL       GOOG      AAPL      GOOG  
Date                                                              
2022-12-23  86.967734  128.974267  86.967734  63814900  17815000  
2022-12-27  86.883365  129.427446  88.645145  69007800  15470900  
2022-12-28  85.727026  127.742842  86.848612  85438400  17879600  
2022-12-29  86.342418  126.087790  86.382121  75703700  18280700  
2022-12-30  86.382124  126.501586  86.714629  77034200  19190300

## <b> Run Backtest </b>

In [4]:
bt = Backtest(data=df_ohlc,
              universe=universe,
              strategy_class=TestStrategy,
              price_column="Close",
              cash=100_000
             )

result = bt.run()

result.head()

Strategy started!
Bought 1239 AAPL and 946 GOOG
Strategy finished!


,BALANCE,POSITIONS_FILLED_TOTAL,POSITIONS_UNFILLED_TOTAL,EQUITY
ITERATION,,,,
2018-01-02,16.910618,99983.089382,0,100000.000000
2018-01-03,16.910618,100795.036133,0,100811.946751
2018-01-04,16.910618,101211.210423,0,101228.121040
2018-01-05,16.910618,102526.010803,0,102542.921421
2018-01-08,16.910618,102558.544518,0,102575.455135


## <b> Backtest Statistics </b>

In [5]:
print("---------------------------")
print("Final balance: %.2f" % bt.strategy.balance)
print("Final equity: %.2f" % bt.strategy.equity)
print("---------------------------")

---------------------------
Final balance: 16.91
Final equity: 241921.38
---------------------------


In [6]:
bt.get_trade_statistics()

{'n_won': 1,
 'n_lost': 0,
 'n_total': 1,
 'win_rate': 1.0,
 'mean_profit': 141921.37510299683,
 'median_profit': 141921.37510299683,
 'mean_profit_win': 141921.37510299683,
 'mean_profit_loss': nan,
 'median_profit_win': 141921.37510299683,
 'median_profit_loss': nan,
 'std_profit': nan,
 'mad_profit': 0.0,
 'skewness': nan,
 'kurtosis': nan,
 'mean_median_gap': 0.0,
 'mean_profit_over_vol': nan,
 'median_profit_over_vol': nan}

In [7]:
bt.get_equity_statistics_dollar()

{'total_profit': 141921.37510299683,
 'mean_profit': 112.81508354769223,
 'median_profit': 256.21557998657227,
 'std_profit': 4002.753777483725,
 'mad_profit': 2424.439759176415,
 'skewness': -0.19350238948255824,
 'kurtosis': 3.0568982899787747,
 'mean_median_gap': -143.40049643888005,
 'mean_profit_over_vol': 0.028184367517757197,
 'median_profit_over_vol': 0.06400982779101606,
 'max_drawdown': -122839.07052612305}

In [8]:
bt.get_equity_statistics_return()

{'total_return': 1.419213751029968,
 'mean_return': 0.0008838018667298779,
 'median_return': 0.001663518439769529,
 'std_return': 0.019027781168376348,
 'mad_return': 0.014074539458795367,
 'skewness': -0.15457401285386202,
 'kurtosis': 4.071141487798516,
 'mean_median_gap': -0.0007797165730396512,
 'mean_return_over_vol': 0.04644797304053152,
 'median_return_over_vol': 0.08742577103704825,
 'max_drawdown': -0.34324007683812896}

In [9]:
bt.tradebook

,ID,STATUS,NOTES,PNL,CREATED_ITERATION,CLOSED_AT_ITERATION,AGE,REASON_CLOSED,POSITIONS_FILLED,POSITIONS_FILLED_TOTAL,POSITIONS_UNFILLED,POSITIONS_UNFILLED_TOTAL,STOP_LOSS,TAKE_PROFIT,MAX_AGE
0,cded0ef2-6aff-4465-9c9b-afc8e5cf1d8e,OPEN,Buy and Hold Trade,141921.375103,2018-01-02,None,1823 days,None,"{'AAPL': {'quantity': 1239, 'value': 158590.74...",241904.464485,{},0,None,None,None


In [10]:
bt.orderbook

,ID,TICKER,ACTION,STATUS,NOTES,TRADE_ID,ITERATION_REQUESTED,QUANTITY_REQUESTED,PRICE_REQUESTED,FEE_REQUESTED,GROSS_TOTAL_REQUESTED,TOTAL_REQUESTED,ITERATION_FILLED,QUANTITY_FILLED,PRICE_FILLED,FEE_FILLED,GROSS_TOTAL_FILLED,TOTAL_FILLED
0,9984d0a9-93ed-4061-bf55-2ef7d5b328c4,AAPL,BUY,FILLED,AAPL buy order,cded0ef2-6aff-4465-9c9b-afc8e5cf1d8e,2018-01-02,1239,40.341885,0.0,49983.595036,49983.595036,2018-01-02,1239,40.341885,0.0,-49983.595036,-49983.595036
1,9aff7463-72fd-42b5-8e13-d187fbde869b,GOOG,BUY,FILLED,GOOG buy order,cded0ef2-6aff-4465-9c9b-afc8e5cf1d8e,2018-01-02,946,52.853588,0.0,49999.494347,49999.494347,2018-01-02,946,52.853588,0.0,-49999.494347,-49999.494347


In [11]:
bt.strategy.positions_filled

{'AAPL': {'quantity': 1239, 'value': np.float64(158590.74277496338)},
 'GOOG': {'quantity': 946, 'value': np.float64(83313.72171020508)}}

In [12]:
bt.strategy.positions_unfilled

{}

In [13]:
bt.strategy.positions_filled

{'AAPL': {'quantity': 1239, 'value': np.float64(158590.74277496338)},
 'GOOG': {'quantity': 946, 'value': np.float64(83313.72171020508)}}